In [2]:
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn import linear_model
import json
import pandas as pd
import pymysql
import os

In [4]:
def CustomParser(data):
    return json.loads(data)

In [34]:
# Read in data and clean JSON column
df = pd.read_csv('data.csv', converters={'data':CustomParser}, header=0)

# Pull JSON values in to its own column
df[sorted(df['data'][0].keys())] = df['data'].apply(pd.Series)

# Drop JSON column
df.drop('data', axis=1, inplace=True)

# Clean Amounts
df['amount'] = df['amount'].astype(float).apply(lambda x: x*-1)

# Convert Dates
df['created_at'] = df['created_at'].astype('datetime64[ns]')

In [35]:
df.head()

,type,created_at,user_id,amount
0,auth,2021-12-30 03:40:03,600,1.00
1,auth,2021-12-30 03:49:49,600,1.00
2,auth,2021-12-31 17:53:21,600,1.95
3,auth,2021-12-30 00:45:48,595,100.00
4,auth,2021-12-30 21:52:42,595,100.00


In [57]:
# Build out date & time dimensions
df['year'] = df['created_at'].dt.year
df['month'] = df['created_at'].dt.month
df['day'] = df['created_at'].dt.day
df['week_of_year'] = df['created_at'].dt.isocalendar().week
df['day_of_week'] = df['created_at'].dt.dayofweek
df['day_name'] = df['created_at'].dt.weekday.map({
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
})
df['date'] = df['created_at'].dt.date
df.head()

,type,created_at,user_id,amount,year,month,day,week_of_year,day_of_week,day_name,date
0,auth,2021-12-30 03:40:03,600,1.00,2021,12,30,52,3,Thursday,2021-12-30
1,auth,2021-12-30 03:49:49,600,1.00,2021,12,30,52,3,Thursday,2021-12-30
2,auth,2021-12-31 17:53:21,600,1.95,2021,12,31,52,4,Friday,2021-12-31
3,auth,2021-12-30 00:45:48,595,100.00,2021,12,30,52,3,Thursday,2021-12-30
4,auth,2021-12-30 21:52:42,595,100.00,2021,12,30,52,3,Thursday,2021-12-30


**Question 1**

We would like to know a few details about the data set in general:
- What is the average transaction value?

In [25]:
df.amount.mean().round(2)

14.11

- What is the average transaction value / user?

In [29]:
df.groupby('user_id')['amount'].mean().round(2)

user_id
97      21.69
101      6.02
116     12.53
117      5.13
123      2.75
        ...  
456      1.99
473     42.29
532     12.99
595    133.33
600      1.32
Name: amount, Length: 103, dtype: float64

- How much are users spending per month?

In [48]:
df.groupby(['month', 'user_id'])['amount'].mean().round(2)

month  user_id
1      117         31.50
       129         40.75
       130         29.08
       139          4.00
       147         25.60
                   ...  
12     455         49.10
       456          1.99
       473         42.29
       595        100.00
       600          1.32
Name: amount, Length: 193, dtype: float64

**Question 2**

Please provide a report showing the number of days a user spent per week

In [56]:
df.groupby(['user_id', 'week_of_year'])['day_of_week'].nunique()

user_id  week_of_year
97       44              4
         45              4
         46              3
101      48              2
         49              1
                        ..
456      52              1
473      52              1
532      52              1
595      52              2
600      52              2
Name: day_of_week, Length: 525, dtype: int64

**Question 3**

Free range: What other insights do you gather from the data? If you could visualize something you find would be amazing

In [4]:
class Database():
    '''
    Generic database class that serves as a foundation for adding
    additional flavors as needed.
    '''
    def __init__(self, host, database, username, password) -> None:
        self._host = host
        self._database = database
        self._username = username
        self._password = password

    @property
    def host(self) -> str:
        return self._host

    @host.setter
    def host(self, value: str) -> None:
        self._host = value

    @property
    def database(self) -> str:
        return self._database

    @database.setter
    def database(self, value: str) -> None:
        self._database = value

    @property
    def username(self) -> str:
        return self._username

    @username.setter
    def username(self, value: str) -> None:
        self._username = value

    @property
    def password(self) -> str:
        return self._password

    @password.setter
    def password(self, value: str) -> None:
        self._password = value

class MySQL(Database):
    '''
    Base MySQL class for interacting with the database.
    '''
    def __init__(self, host, database, username, password, port: str = '3306'):
        super().__init__(host, database, username, password)
        self._port = port
        self.engine = create_engine(f"mysql+pymysql://{self.username}:{self.password}@{self.host}/{self.database}")

    @property
    def port(self) -> str:
        return self._port

    @port.setter
    def port(self, value: str) -> None:
        self._port = value
        
    def load_df(self, df: pd.DataFrame, table: str) -> None:
        conn = self.engine.connect()
        try:
            df.to_sql(table.lower(), conn, if_exists='append', index=False)
        finally:
            conn.close()
            
    def sql_to_df(self, sql: str) -> pd.DataFrame:
        conn = self.engine.connect()
        df = pd.read_sql(sql, conn)
        conn.close()
        return df
    
    def run_sql(self, sql: str) -> None:
        conn = self.engine.connect()
        conn.execute(sql)
        conn.close()

In [5]:
load_dotenv('.env')
ms = MySQL(
    host=os.getenv("MYSQL_HOST"),
    port=os.getenv("MYSQL_PORT"),
    database=os.getenv("MYSQL_SCH"),
    username=os.getenv("MYSQL_USER"),
    password=os.getenv("MYSQL_PASS")
)

In [ ]:
ms.load_df(df, 'transactions')

In [6]:
# Pull list of users to loop through
users = pd.read_sql("SELECT user_id FROM users_spend GROUP BY user_id HAVING COUNT(month) > 1 ORDER BY 1", ms.engine)
users = users["user_id"].unique()

# Pull training data set
training = pd.read_sql("SELECT * FROM users_spend WHERE user_id IN (SELECT user_id FROM users_spend GROUP BY user_id HAVING COUNT(month) > 1)", ms.engine)
training['month'] = pd.to_datetime(training['month'])

In [7]:
for user in users:
    temp_df = training[training['user_id']==user].copy(deep=True)
    temp_df.set_index('month', inplace=True)
    temp_df.insert(0, 'id', range(0, 0 + len(temp_df)))
    temp_df.drop('user_id', axis=1, inplace=True)
    x = temp_df['id'].values
    y = temp_df['amount'].values
    x = x.reshape(-1,1)
    model = linear_model.LinearRegression().fit(x, y)
    linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
    print(f"user_id: {user} prediction: {0.00 if model.predict([[2]])[0] < 0 else model.predict([[2]])[0]:.2f}")

user_id: 116 prediction: 54.98
user_id: 117 prediction: 304.66
user_id: 123 prediction: 0.00
user_id: 125 prediction: 4.96
user_id: 127 prediction: 2357.44
user_id: 129 prediction: 2224.70
user_id: 130 prediction: 4450.40
user_id: 131 prediction: 68.46
user_id: 135 prediction: 0.00
user_id: 137 prediction: 4178.95
user_id: 139 prediction: 533.63
user_id: 144 prediction: 0.00
user_id: 150 prediction: 192.63
user_id: 151 prediction: 0.00
user_id: 153 prediction: 217.06
user_id: 154 prediction: 0.00
user_id: 155 prediction: 650.67
user_id: 164 prediction: 108.07
user_id: 167 prediction: 527.72
user_id: 168 prediction: 23.43
user_id: 172 prediction: 0.00
user_id: 174 prediction: 14.75
user_id: 175 prediction: 978.53
user_id: 178 prediction: 950.73
user_id: 180 prediction: 5.81
user_id: 181 prediction: 0.00
user_id: 184 prediction: 719.06
user_id: 185 prediction: 0.71
user_id: 196 prediction: 0.00
user_id: 200 prediction: 416.23
user_id: 201 prediction: 0.00
user_id: 205 prediction: 7.14
us